In [1]:
# src/preprocessing.py
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append("..")
from config import RAW_DATA_DIR, PROCESSED_DATA_DIR, TARGET_COLUMN, RANDOM_STATE

def create_binary_target(df):
    """
    Convert loan_status to binary classification
    0 = Non-Default (Fully Paid)
    1 = Default (Charged Off)
    
    Excludes ongoing loans (Current, Late, In Grace Period)
    """
    print("\n" + "="*60)
    print("CREATING BINARY TARGET")
    print("="*60)
    
    # Define completed loan statuses
    non_default_statuses = [
        'Fully Paid',
        'Does not meet the credit policy. Status:Fully Paid'
    ]
    
    default_statuses = [
        'Charged Off',
        'Does not meet the credit policy. Status:Charged Off',
        'Default'
    ]
    
    # Filter to completed loans only
    completed_mask = df['loan_status'].isin(non_default_statuses + default_statuses)
    df_completed = df[completed_mask].copy()
    
    print(f"\nOriginal dataset: {len(df):,} rows")
    print(f"Completed loans: {len(df_completed):,} rows ({len(df_completed)/len(df)*100:.1f}%)")
    print(f"Excluded (ongoing): {len(df) - len(df_completed):,} rows")
    
    # Create binary target
    df_completed['default'] = df_completed['loan_status'].isin(default_statuses).astype(int)
    
    # Show distribution
    print(f"\n{'='*60}")
    print("CLASS DISTRIBUTION")
    print("="*60)
    print(f"\nNon-Default (0): {(df_completed['default']==0).sum():,} ({(df_completed['default']==0).mean()*100:.2f}%)")
    print(f"Default (1):     {(df_completed['default']==1).sum():,} ({(df_completed['default']==1).mean()*100:.2f}%)")
    
    imbalance_ratio = (df_completed['default']==0).sum() / (df_completed['default']==1).sum()
    print(f"\nImbalance Ratio: 1:{imbalance_ratio:.2f}")
    print(f"Default Rate: {df_completed['default'].mean()*100:.2f}%")
    
    return df_completed


def select_features(df):
    """
    Select most important features for loan default prediction
    Reduces from 145 to ~25 relevant features
    """
    print("\n" + "="*60)
    print("FEATURE SELECTION")
    print("="*60)
    
    # Core features for loan default prediction
    selected_features = [
        # Loan characteristics
        'loan_amnt',           # Loan amount requested
        'funded_amnt',         # Amount funded by investors
        'int_rate',            # Interest rate
        'installment',         # Monthly payment
        'term',                # 36 or 60 months
        'grade',               # Loan grade (A-G)
        'sub_grade',           # Loan sub-grade
        
        # Borrower employment & income
        'emp_length',          # Employment length
        'annual_inc',          # Annual income
        'verification_status', # Income verification status
        
        # Housing
        'home_ownership',      # RENT, OWN, MORTGAGE, OTHER
        
        # Credit profile
        'dti',                 # Debt-to-income ratio
        'delinq_2yrs',        # Delinquencies in past 2 years
        'earliest_cr_line',    # Date of earliest credit line
        'inq_last_6mths',     # Credit inquiries in last 6 months
        'open_acc',            # Number of open credit accounts
        'pub_rec',             # Number of derogatory public records
        'revol_bal',           # Total revolving credit balance
        'revol_util',          # Revolving line utilization rate
        'total_acc',           # Total number of credit lines
        
        # Geographic
        'addr_state',          # Borrower state
        
        # Loan purpose
        'purpose',             # Purpose of loan
        
        # Target
        'loan_status',         # Original status
        'default',             # Binary target (will be created)
    ]
    
    # Only select features that exist in the dataframe
    existing_features = [f for f in selected_features if f in df.columns]
    
    print(f"\nSelected {len(existing_features)} features from {len(df.columns)} total")
    
    df_selected = df[existing_features].copy()
    
    return df_selected


def handle_missing_values(df):
    """
    Handle missing values in the dataset
    """
    print("\n" + "="*60)
    print("HANDLING MISSING VALUES")
    print("="*60)
    
    # Check missing values before
    missing_before = df.isnull().sum().sum()
    print(f"\nTotal missing values: {missing_before:,}")
    
    # Show features with missing values
    missing_df = pd.DataFrame({
        'Missing': df.isnull().sum(),
        'Percent': (df.isnull().sum() / len(df)) * 100
    }).sort_values('Percent', ascending=False)
    
    print(f"\nFeatures with missing values (>0%):")
    print(missing_df[missing_df['Missing'] > 0])
    
    # Strategy for each feature type:
    
    # 1. Drop rows with missing target (shouldn't happen, but just in case)
    if 'default' in df.columns:
        df = df.dropna(subset=['default'])
    
    # 2. Numerical features - impute with median
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        if df[col].isnull().any():
            median_val = df[col].median()
            df[col].fillna(median_val)
            print(f"  - Imputed {col} with median: {median_val:.2f}")
    
    # 3. Categorical features - impute with mode or 'Unknown'
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if col not in ['loan_status', 'default'] and df[col].isnull().any():
            mode_val = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
            df[col].fillna(mode_val, inplace=True)
            print(f"  - Imputed {col} with mode: {mode_val}")
    
    # Check missing values after
    missing_after = df.isnull().sum().sum()
    print(f"\nMissing values after imputation: {missing_after:,}")
    
    return df


def clean_features(df):
    """
    Clean and standardize feature formats
    """
    print("\n" + "="*60)
    print("CLEANING FEATURES")
    print("="*60)
    
    df_clean = df.copy()
    
    # 1. Clean interest rate (remove % sign, convert to float)
    if 'int_rate' in df_clean.columns:
     df_clean['int_rate'] = df_clean['int_rate'].astype(str).str.replace('%', '')
    df_clean['int_rate'] = pd.to_numeric(df_clean['int_rate'], errors='coerce')
    print("  ✓ Cleaned int_rate: removed % sign")
    
    # 2. Clean revolving utilization (remove % sign)
    if 'revol_util' in df_clean.columns:
        df_clean['revol_util'] = df_clean['revol_util'].astype(str).str.replace('%', '')
        df_clean['revol_util'] = pd.to_numeric(df_clean['revol_util'], errors='coerce')
        print("  ✓ Cleaned revol_util: removed % sign")
    
    # 3. Convert term to numeric (extract months)
    if 'term' in df_clean.columns:
        df_clean['term'] = df_clean['term'].str.extract(r'(\d+)').astype(int)  # Added 'r' before string
        print("  ✓ Cleaned term: extracted months as integer")
    
    # 4. Convert employment length to numeric
    if 'emp_length' in df_clean.columns:
        emp_map = {
            '< 1 year': 0,
            '1 year': 1,
            '2 years': 2,
            '3 years': 3,
            '4 years': 4,
            '5 years': 5,
            '6 years': 6,
            '7 years': 7,
            '8 years': 8,
            '9 years': 9,
            '10+ years': 10
        }
        df_clean['emp_length'] = df_clean['emp_length'].map(emp_map)
        print("  ✓ Cleaned emp_length: converted to years (0-10)")
    
    # 5. Convert earliest_cr_line to credit history length
    if 'earliest_cr_line' in df_clean.columns:
        df_clean['earliest_cr_line'] = pd.to_datetime(df_clean['earliest_cr_line'], errors='coerce')
        df_clean['credit_history_years'] = (pd.Timestamp.now() - df_clean['earliest_cr_line']).dt.days / 365.25
        df_clean = df_clean.drop('earliest_cr_line', axis=1)
        print("  ✓ Converted earliest_cr_line to credit_history_years")
    
    return df_clean


def preprocess_pipeline(input_path=None, output_path=None):
    """
    Full preprocessing pipeline
    """
    from src.data_loader import load_data
    
    print("\n" + "="*60)
    print("STARTING PREPROCESSING PIPELINE")
    print("="*60)
    
    # 1. Load data
    df = load_data(input_path)
    
    # 2. Create binary target
    df = create_binary_target(df)
    
    # 3. Select features
    df = select_features(df)
    
    # 4. Clean features
    df = clean_features(df)
    
    # 5. Handle missing values
    df = handle_missing_values(df)
    
    # 6. Final check
    print("\n" + "="*60)
    print("PREPROCESSING COMPLETE")
    print("="*60)
    print(f"\nFinal dataset shape: {df.shape}")
    print(f"Features: {len(df.columns)}")
    print(f"Samples: {len(df):,}")
    print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
    
    # 7. Save processed data
    if output_path is None:
        output_path = PROCESSED_DATA_DIR / "loan_data_processed.csv"
    
    df.to_csv(output_path, index=False)
    print(f"\n✓ Saved processed data to: {output_path}")
    
    return df


if __name__ == "__main__":
    # Run the full pipeline
    df_processed = preprocess_pipeline()
    
    # Show final summary
    print("\n" + "="*60)
    print("FINAL DATASET SUMMARY")
    print("="*60)
    print(f"\nShape: {df_processed.shape}")
    print(f"\nFirst few rows:")
    print(df_processed.head())
    print(f"\nData types:")
    print(df_processed.dtypes)
    print(f"\nTarget distribution:")
    print(df_processed['default'].value_counts())

/Users/theboybrey/dev/workspace/cscd/observatory/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



STARTING PREPROCESSING PIPELINE
Loading data from: /Users/theboybrey/dev/workspace/cscd/observatory/notebooks/../data/raw/loan.csv
Data loaded: 2260668 rows, 145 columns

CREATING BINARY TARGET

Original dataset: 2,260,668 rows
Completed loans: 1,306,387 rows (57.8%)
Excluded (ongoing): 954,281 rows

CLASS DISTRIBUTION

Non-Default (0): 1,043,940 (79.91%)
Default (1):     262,447 (20.09%)

Imbalance Ratio: 1:3.98
Default Rate: 20.09%

FEATURE SELECTION

Selected 24 features from 146 total

CLEANING FEATURES
  ✓ Cleaned int_rate: removed % sign
  ✓ Cleaned revol_util: removed % sign
  ✓ Cleaned term: extracted months as integer
  ✓ Cleaned emp_length: converted to years (0-10)
  ✓ Converted earliest_cr_line to credit_history_years

HANDLING MISSING VALUES

Total missing values: 76,832


/var/folders/41/ntbk682n529_2tx7mfmjrj7r0000gn/T/ipykernel_14808/4264806503.py:215: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean['earliest_cr_line'] = pd.to_datetime(df_clean['earliest_cr_line'], errors='coerce')



Features with missing values (>0%):
                      Missing   Percent
emp_length              75491  5.778609
revol_util                850  0.065065
dti                       312  0.023883
inq_last_6mths             30  0.002296
delinq_2yrs                29  0.002220
total_acc                  29  0.002220
pub_rec                    29  0.002220
open_acc                   29  0.002220
credit_history_years       29  0.002220
annual_inc                  4  0.000306
  - Imputed emp_length with median: 6.00
  - Imputed annual_inc with median: 65000.00
  - Imputed dti with median: 17.60
  - Imputed delinq_2yrs with median: 0.00
  - Imputed inq_last_6mths with median: 0.00
  - Imputed open_acc with median: 11.00
  - Imputed pub_rec with median: 0.00
  - Imputed revol_util with median: 52.30
  - Imputed total_acc with median: 23.00
  - Imputed credit_history_years with median: 25.60


/var/folders/41/ntbk682n529_2tx7mfmjrj7r0000gn/T/ipykernel_14808/4264806503.py:154: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = df.select_dtypes(include=['object']).columns



Missing values after imputation: 76,832

PREPROCESSING COMPLETE

Final dataset shape: (1306387, 24)
Features: 24
Samples: 1,306,387

Memory usage: 698.90 MB

✓ Saved processed data to: /Users/theboybrey/dev/workspace/cscd/observatory/notebooks/../data/processed/loan_data_processed.csv

FINAL DATASET SUMMARY

Shape: (1306387, 24)

First few rows:
     loan_amnt  funded_amnt  int_rate  installment  term grade sub_grade  \
100      30000        30000     22.35      1151.16    36     D        D5   
152      40000        40000     16.14       975.71    60     C        C4   
170      20000        20000      7.56       622.68    36     A        A3   
186       4500         4500     11.31       147.99    36     B        B3   
215       8425         8425     27.27       345.18    36     E        E5   

     emp_length  annual_inc verification_status  ... open_acc  pub_rec  \
100         5.0    100000.0     Source Verified  ...     11.0      1.0   
152         0.0     45000.0            Verifie